### Müllabfuhr - Wahl von 5 Pfaden

[Aufgabe](muellabfuhr.pdf) - [Lösungshinweise](muellabfuhrL.pdf)

In [636]:
%%time
'''
6: 190 ms, 7: 22.6 s, 8: 3min 3s     
'''
inf = float('inf')
nr = 0
f = open('./beispieldaten/muellabfuhr'+str(nr)+'.txt')
anzV, anzE = [int(x) for x in f.readline().split()]

G = [[inf]*anzV for _ in range(anzV)]
V = set()
E = set()
for i in range(anzE):
    von, bis, km = [int(x) for x in f.readline().split()]
    G[von][bis]=km
    G[bis][von]=km
    V.add(von)
    V.add(bis)
    
    if von < bis:
        E.add((von,bis))
    else:
        E.add((bis,von))
for i in range(anzV):
    G[i][i]=0
    
def floyd(c):
    n = len(c)
    d = [[0]*n for j in range(n)]   # Matrix D
    p = [[0]*n for j in range(n)]   # Matrix P
    for i in range(n):
        for j in range(n):
            d[i][j] = c[i][j]
            p[i][j] = i

    for k in range(n):
        for i in range(n):
            for j in range(n):
                tmp = d[i][k] + d[k][j]
                if tmp < d[i][j]:
                    d[i][j] = tmp
                    p[i][j] = p[k][j]
    return d, p

def getPath(p, i, j):
    if i == j: return (i,)
    return getPath(p,i,p[i][j]) + (j,)

def printMatrix(a):
    for i in range(len(a)):
        for j in range(len(a)):
            if a[i][j] == inf:
                print("{:4}".format("."),end='')
            else:
                print("{:<4}".format(a[i][j]),end='')
        print()
    print()
    
D, P = floyd(G)


# die Matrizen D und P werden als globale Variablen vorausgesetzt.
def dist(v1, v2):
    '''
    returns: kürzeste Distanz zwischen v1 und v2
    '''
    return D[v1][v2]

def sPath(v1, v2):
    '''
    returns: einen kürzesten Weg zwischen v1 und v2
    '''
    return getPath(P,v1,v2)

def match(U):
    '''
    returns: greedy match
    '''
    if len(U) == 0:
        return []
    v = U[0]
    best = 1
    best_val = dist(U[0],U[1])
    for i in range(2,len(U)):
        if U[i] not in G[U[0]]:
            val = dist(U[0],U[i])
            if val < best_val:
                best_val = val
                best = i
    e = (U[0],U[best])
    U1 = U.copy()
    U1.pop(best)
    U1.pop(0)
    return match(U1) + [e]

def abschnitt(p,i,j):
    '''
    returns: Kosten für den Weg 0 - p[i]-p[j] - 0
    '''
    pcost = 0
    for k in range(i,j):

        pcost += G[p[k]][p[k+1]]
    return dist(0,p[i]) + dist(p[j],0) + pcost

def aufteilung(path,n):
    tmp = []
    i = 0
    j = 0
    for k in range(5):
        while j < len(path) and (abschnitt(path,i,j)) < n:
            j+=1
        tmp.append((i,j-1))
        i=j-1
        j=i+1
    return tmp

CPU times: total: 0 ns
Wall time: 0 ns


#### Einlesen des Graphen mit Adjazenzsets

Wir haben unter Umständen Mehrfachkanten, deswegen 

In [637]:
 
f = open('./beispieldaten/muellabfuhr'+str(nr)+'.txt')
anzV, anzE = [int(x) for x in f.readline().split()]
G = {x:dict() for x in range(anzV)}
for _ in range(anzE):
    v1, v2, cost = f.readline().split()
    v1, v2, cost = int(v1), int(v2), int(cost)
    G[v1][v2] = cost
    G[v2][v1] = cost
 

#Knoten mit ungeradem Grad bestimmen und ein gieriges Matching durchführen

U = []
for v in G:
    if len(G[v]) % 2 != 0:
        U.append(v)
len(U)

meta = match(U)
meta, len(meta)

([(6, 9), (2, 4)], 2)

Metakanten einführen. Dazu Metaknoten einführen, weil unsere Modellierung keine doppelten Kanten kann


    

In [638]:
hilf = 10000
for e in meta:
    v1, v2 = e
    G[hilf] = dict()
    G[v1][hilf] = G[v2][hilf] = dist(v1,v2)
    G[hilf][v2] = G[hilf][v1] = 0
    hilf+=1
    
 
degrees = {x:len(G[x]) for x in G} 

In [639]:
# Kontrolle, ob alle Knoten geraden Grad haben
for v in G:
    if len(G[v]) % 2 != 0:
        print(v)

In [640]:
from copy import deepcopy
GE = deepcopy(G)
degrees = {x:len(GE[x]) for x in GE} 
def dfs(v):
    while degrees[v] != 0:
        next = list(GE[v])[0]  # die erste noch vorhandene Kante
        del(GE[v][next])     # wird aus dem Graphen gelöscht
        del(GE[next][v])     # von beiden Seiten
        degrees[v]-=1         # der Grad der Knoten v und next
        degrees[next]-=1      # wird erniedrigt
        dfs(next)
    path.append(v)  
path = []
dfs(0)
path.reverse()                # beim undirected eulercircle unnötig
print(path)    

[0, 2, 1, 8, 0, 4, 3, 2, 10001, 4, 5, 6, 7, 8, 9, 10000, 6, 0]


In [645]:
visited = {v:set() for v in G}            # die schon besuchten Kanten
degrees = {x:len(G[x]) for x in G} 
path = []

def dfs(v):
    while degrees[v] != 0:
                 
        for w in G[v]:
            if w not in visited[v]:
                next = w      # die nächste noch unbesuchte Kante
                break
                
        visited[v].add(next)  # markiere Kante (v,next) als besucht
        visited[next].add(v)

        degrees[v]-=1         # der Grad der Knoten v und next
        degrees[next]-=1      # wird erniedrigt
        dfs(next)
    path.append(v)  
dfs(0)
path.reverse()                # beim undirected eulercircle unnötig
print(path)

[0, 2, 1, 8, 0, 4, 3, 2, 10001, 4, 5, 6, 7, 8, 9, 10000, 6, 0]


In [642]:
def eliminateMeta(path):
    for i in range(len(path)-1,-1,-1):
        if path[i] >= 10000:
            zwischen = list(sPath(path[i-1],path[i+1]))
            path = (path[:i]+zwischen[1:-1]+path[i+1:])
    return path
            
path = eliminateMeta(path)
print(path)

[0, 2, 1, 8, 0, 4, 3, 2, 0, 4, 5, 6, 7, 8, 9, 8, 0, 6, 0]


In [644]:
mincost = 0
maxcost = abschnitt(path,0,len(path)-1)


fertig = False
while not fertig:
    mitte = (mincost+maxcost)//2
    au = aufteilung(path,mitte)
    lastIndex = au[-1][1]
    if lastIndex == len(path)-1:
        maxcost = mitte-1
    else:
        mincost = mitte+1
    if mincost >= maxcost:
        break
   
print(mitte)
aufteilung(path,mitte),len(path)

7


([(0, 5), (5, 9), (9, 13), (13, 18), (18, 18)], 19)